In [20]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import json

In [21]:
zones = gpd.read_file("../data/curated/vic_localities/vic_localities.shp")
zones.head()

,LC_PLY_PID,LOC_PID,DT_CREATE,LOC_NAME,LOC_CLASS,STATE,geometry
0,lcp6229215ba53f,locb0dcb52a6b55,2021-06-24,Abbeyard,Gazetted Locality,VIC,"POLYGON ((146.81721 -37.09735, 146.81729 -37.0..."
1,lcp386f2bcf9bce,locb9872f35df41,2021-06-24,Abbotsford,Gazetted Locality,VIC,"POLYGON ((145.00235 -37.80723, 145.00350 -37.8..."
2,lcp122c942a8fc9,loc8123ed12ea8d,2021-06-24,Aberfeldie,Gazetted Locality,VIC,"POLYGON ((144.89830 -37.76465, 144.89790 -37.7..."
3,lcp9f50bd795d16,loc1eb4a229104a,2021-06-24,Aberfeldy,Gazetted Locality,VIC,"POLYGON ((146.39447 -37.71008, 146.39405 -37.7..."
4,lcp59bf69caacb0,locb17be87767e1,2021-06-24,Acheron,Gazetted Locality,VIC,"POLYGON ((145.75030 -37.24313, 145.75036 -37.2..."


In [22]:
# express location in latitude and longitude and unify the format
zones['geometry'] = zones['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
zones.rename(columns={'LOC_NAME':'suburb'},inplace = True)
zones['suburb'] = zones['suburb'].str.upper()

In [23]:
#convert to json
geoJSON = zones[['suburb', 'geometry']].to_json()
print(geoJSON[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"suburb": "ABBEYARD"}, "geometry": {"type": "Polygon", "coordinates": [[[146.8172166420058, -37.09733585571731], [146.81729332194828, -37.09771416572554], [146.81740626189682, -37.09798946573587], [146.817497691


In [24]:
map_apa = folium.Map(location=[ -37.8136, 144.9631], tiles="Stamen Terrain", zoom_start=10)


In [25]:
# read data
apa = pd.read_csv('../data/curated/match_apartment22_s.csv')
house = pd.read_csv('../data/curated/match_house22_s.csv')

In [26]:
#calculate average house and apartment price in each suburb
agg_df = apa[['suburb', 'price']].groupby('suburb').mean()
agg_df = agg_df.nlargest(50,'price')
agg_df['suburb'] = agg_df.index
agg_df.head()

,price,suburb
suburb,,
WARRAGUL,1434.000000,WARRAGUL
MELBOURNE,838.571429,MELBOURNE
ST KILDA,831.805556,ST KILDA
SOUTH MELBOURNE,720.000000,SOUTH MELBOURNE
PORT MELBOURNE,713.863636,PORT MELBOURNE


In [27]:
house_df = house[['suburb', 'price']].groupby('suburb').mean()
house_df = house_df.nlargest(50,'price')
house_df['suburb'] = house_df.index
house_df.head()

,price,suburb
suburb,,
SYDENHAM,1746.666667,SYDENHAM
TOORAK,1543.888889,TOORAK
KEW,1510.000000,KEW
HAMPTON,1234.166667,HAMPTON
HIGHTON,1224.739130,HIGHTON


In [28]:
agg_df['price'].describe()

count      50.000000
mean      537.449760
std       159.118282
min       427.500000
25%       456.964286
50%       491.277778
75%       537.708333
max      1434.000000
Name: price, dtype: float64

In [29]:

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=agg_df, # data source
    columns=['suburb','price'], # the columns required
    key_on='properties.suburb', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color = 'silver',
    highlight=True,
    line_weight= 0.5,
    legend_name='mean price'
)

In [30]:
c.add_to(map_apa)


In [31]:
map_apa.save('../plots/apa_priceMap.html')

In [32]:
map_house = folium.Map(location=[ -37.8136, 144.963], tiles="Stamen Terrain", zoom_start=10)

In [33]:
h = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=house_df, # data source
    columns=['suburb','price'], # the columns required
    key_on='properties.suburb', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color = 'silver',
    highlight=True,
    line_weight= 0.5,
    legend_name='mean price'
)

In [34]:
h.add_to(map_house)
map_house.save('../plots/house_priceMap.html')